In [ ]:
import torch
import torch.nn
from torch.autograd.gradcheck import zero_gradients
import torch.nn.functional as F
import torchvision.models as models
from PIL import Image
from torchvision import transforms
from torchvision.utils import save_image
import json
import numpy as np
import requests, io
import matplotlib.pyplot as plt
from torch.autograd import Variable
%matplotlib inline


In [ ]:
#download and load pretrained inceptionv3 model
inceptionv3 = models.inception_v3(pretrained=True) 
inceptionv3.eval();

In [ ]:
# Upload target image
img = Image.open("Clean44.jpg")

In [ ]:
#Set mean and std deviation
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

preprocess = transforms.Compose([
                transforms.Resize((299,299)),  
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

In [ ]:
#preprocess image
image_tensor = preprocess(img) 
# add batch dimension.  C X H X W ==> B X C X H X W
image_tensor = image_tensor.unsqueeze(0)

In [ ]:
#convert tensor into a variable
img_variable = Variable(image_tensor, requires_grad=True) 

In [ ]:
#Classify image using pre-trained inceptionv3 model
output = inceptionv3.forward(img_variable)
#get an index(class number) of a largest element
label_idx = torch.max(output.data,1)[1][0]   
print(label_idx.item())

In [ ]:
#Upload JSON file containing class labels
with open('labels.json') as f:
    labels_json = json.load(f)
labels = {int(idx):label for idx, label in labels_json.items()}

In [ ]:
#Assign label to predicted image
x_pred = labels[label_idx.item()]
print(x_pred)

In [ ]:
#get probability dist over classes
output_probs = F.softmax(output, dim=1)
x_pred_prob =  ((torch.max(output_probs.data, 1)[0][0]) * 100)
input_prob = x_pred_prob.item()
print (input_prob)

In [ ]:
# Change y_true to input image class  
y_true = Variable( torch.LongTensor([285]), requires_grad=False)
#BIM hyperparameters
epsilon = 0.10
num_steps = 20
alpha = 0.025


In [ ]:
for i in range(num_steps):
  zero_gradients(img_variable)                       #flush gradients
  output = inceptionv3.forward(img_variable)         #perform forward pass
  loss = torch.nn.CrossEntropyLoss()
  loss_cal = loss(output, y_true)
  loss_cal.backward(retain_graph=True)
  x_grad = alpha * torch.sign(img_variable.grad.data)   # as per the formula
  adv_temp = img_variable.data + x_grad                 #add perturbation to img_variable which also contains perturbation from previous iterations
  total_grad = adv_temp - image_tensor                  #total perturbation
  total_grad = torch.clamp(total_grad, -epsilon, epsilon)
  x_adv = image_tensor + total_grad                      #add total perturbation to the original image
  img_variable.data = x_adv

#final adversarial example can be accessed at- img_variable.data

In [ ]:
def visualize(x, x_adv, x_grad, epsilon, clean_pred, adv_pred, clean_prob, adv_prob):
    
   
    
    x = x.squeeze(0)     #remove batch dimension # B X C H X W ==> C X H X W
    x = x.mul(torch.FloatTensor(std).view(3,1,1)).add(torch.FloatTensor(mean).view(3,1,1)).numpy()#reverse of normalization op- "unnormalize"
    x = np.transpose( x , (1,2,0))   # C X H X W  ==>   H X W X C
    x = np.clip(x, 0, 1)
    
    x_adv = x_adv.squeeze(0)
    x_adv = x_adv.mul(torch.FloatTensor(std).view(3,1,1)).add(torch.FloatTensor(mean).view(3,1,1)).numpy()#reverse of normalization op
    x_adv = np.transpose( x_adv , (1,2,0))   # C X H X W  ==>   H X W X C
    x_adv = np.clip(x_adv, 0, 1)
    
    x_grad = x_grad.squeeze(0).numpy()
    x_grad = np.transpose(x_grad, (1,2,0))
    x_grad = np.clip(x_grad, 0, 1)
    
    figure, ax = plt.subplots(1,3, figsize=(18,8))
    ax[0].imshow(x)
    ax[0].set_title('Original Image', fontsize=20)
    
    
    ax[1].imshow(x_grad)
    ax[1].set_title('Perturbation', fontsize=20)
    ax[1].set_yticklabels([])
    ax[1].set_xticklabels([])
    ax[1].set_xticks([])
    ax[1].set_yticks([])

    
    ax[2].imshow(x_adv)
    ax[2].set_title('Adversarial Example', fontsize=20)
    
    ax[0].axis('off')
    ax[2].axis('off')

    ax[0].text(1.1,0.5, "+{}*".format(round(epsilon,3)), size=15, ha="center", 
             transform=ax[0].transAxes)
    
    ax[0].text(0.5,-0.13, "Prediction: {}\n Probability: {}".format(clean_pred, clean_prob), size=15, ha="center", 
         transform=ax[0].transAxes)
    
    ax[1].text(1.1,0.5, " = ", size=15, ha="center", transform=ax[1].transAxes)

    ax[2].text(0.5,-0.13, "Prediction: {}\n Probability: {}".format(adv_pred, adv_prob), size=15, ha="center", 
         transform=ax[2].transAxes)
    

    plt.show()


In [ ]:
output_adv = inceptionv3.forward(img_variable)
y_add = torch.max(output_adv.data, 1)[1][0]
x_adv_pred = labels[y_add.item()]  #classify adversarial example
output_adv_probs = F.softmax(output_adv, dim=1)
x_adv_pred_prob =  ((torch.max(output_adv_probs.data, 1)[0][0]) * 100)
visualize(image_tensor, img_variable.data, total_grad, epsilon, x_pred,x_adv_pred, x_pred_prob,  x_adv_pred_prob) 